In [5]:
import pandas as pd
import numpy as np

In [13]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, GlobalMaxPool1D, Input
from keras.layers import Convolution1D, Conv1D
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.preprocessing import sequence
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.initializers import Constant

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


from keras import Model, models
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

import spacy

In [7]:
data = pd.read_csv('reviews_cleaned.csv')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], 
                                                    random_state = 42, test_size =0.05, stratify= data['sentiment'])

In [9]:
max_features = 50000
maxlen = 5000
embedding_dims = 300



tokenizer = Tokenizer(num_words=max_features, oov_token = '<UNK>')

tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

list_tokenized_train = tokenizer.texts_to_sequences(X_train)

maxlen = max([len(x) for x in list_tokenized_train])

list_tokenized_test = tokenizer.texts_to_sequences(X_test)


x_train = sequence.pad_sequences(list_tokenized_train, padding = 'post', 
                                 truncating = 'post', maxlen=maxlen)

x_test = sequence.pad_sequences(list_tokenized_test, padding = 'post', 
                                 truncating = 'post', maxlen=maxlen)


In [ ]:
# Creating a embedding using glove

embeddings_index = dict()

with open('glove.6B.300d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        

num_words = min(num_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dims))

for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector




In [ ]:
kernel_sizes=[2, 3, 4, 5, 6]
class_num = 1
last_activation='sigmoid' 
sequence_length = data.shape[1]

batch_size = 64
embedding_dims = 300
epochs = 10
max_features = num_words

convs = []
max_poolings = []


for kernel_size in kernel_sizes:
    convs.append(Conv1D(256, kernel_size, activation='relu'))
    max_poolings.append(GlobalMaxPooling1D())

inputs = Input(shape=(x_train.shape[1],), dtype='int32')

embedding = Embedding(max_features, embedding_dims, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inputs)

convs2 = []
for i in range(len(kernel_sizes)):
    c = convs[i](embedding)
    c = max_poolings[i](c)
    convs2.append(c)
x = Concatenate()(convs2)
output = Dense(class_num, activation=last_activation)(x)


model = Model(inputs=inputs, outputs=output)

model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [10]:
model = models.load_model('model_textcnn_imbd.h5')


In [11]:
result = model.predict(x_test)

In [14]:
y_pred = (result > 0.5)

print(classification_report(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

              precision    recall  f1-score   support

       False       0.92      0.93      0.92      1240
        True       0.93      0.92      0.93      1260

    accuracy                           0.92      2500
   macro avg       0.92      0.92      0.92      2500
weighted avg       0.92      0.92      0.92      2500

Confusion matrix:


array([[1151,   89],
       [  99, 1161]], dtype=int64)

In [15]:
example = ["Joaquin Phoenix gives a tour de force performance, fearless and stunning in its emotional depth and physicality. \
            It's impossible to talk about this without referencing Heath Ledger's Oscar-winning performance from The Dark Knight, \
            widely considered the definitive live-action portrayal of the Joker, so let's talk about it. \
            The fact is, everyone is going to be stunned by what Phoenix accomplishes, because it's what many thought impossible \
            a portrayal that matches and potentially exceeds that of The Dark Knight's Clown Prince of Crime"]


In [16]:
tokenized_example = tokenizer.texts_to_sequences(example)

In [17]:
example_test = sequence.pad_sequences(tokenized_example, padding = 'post', 
                                         truncating = 'post', maxlen=maxlen)

In [18]:
example_result = model.predict(example_test)

In [19]:
example_result

array([[0.9999981]], dtype=float32)

In [20]:
if example_result > 0.5:
    print(f'The review is positive')
else:
    print('The review is negative')

The review is positive
